In [ ]:
import numpy as np
import math
import cv2
import os
from scipy.signal import convolve2d

os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/part4', exist_ok=True)

def build_kernel(r):
    kernel = np.zeros((2*r+1,2*r+1))
    for i in range(2*r+1):
        for j in range(2*r+1):
            dis = math.sqrt((i-r)**2 + (j-r)**2)
            if(dis<=r-0.5):
                kernel[i][j] = 1
            elif(r-0.5 < dis < r+0.5):
                kernel[i][j] = r + 0.5 - dis
    return kernel

def pad_image(img):
    m, n = img.shape[:2]
    return cv2.copyMakeBorder(img, 0, m, 0, n, cv2.BORDER_REFLECT)

def pad_kernel(kernel, m, n):
    a,b = kernel.shape[:2]
    padded_kernel = np.zeros((2*m,2*n))
    padded_kernel[:a,:b] = kernel
    return padded_kernel

def apply_freq_domain_blurring(image, r):
    m,n = image.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_image = pad_image(image)

    freq_img = np.fft.fft2(padded_image)

    freq_kernel = np.fft.fft2(kernel)

    freq_blurred_img = freq_img * freq_kernel
    blurred_img = np.fft.ifft2(freq_blurred_img).real
    blurred_img = blurred_img[0:m,0:n]
    return blurred_img

def degrade_image(img, r):
    imax = np.max(img)
    bl_img = apply_freq_domain_blurring(img, r)

    a = 0.05*imax
    b = 0.1*imax
    mean = (a + b) / 2.0
    stddev = (b - a) / 6.0

    noise = np.random.normal(mean, stddev, img.shape)

    d_img= bl_img + noise
    d_img_disp = np.uint8(cv2.normalize(d_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/degraded_image_{r}.png',d_img_disp)
    return d_img


def inverse_filtering(img, r):
    m,n = img.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_image = pad_image(img)

    freq_img = np.fft.fft2(padded_image)

    H = np.fft.fft2(kernel)
    H[H<1e-6] = 1e-6

    freq_clean_img = freq_img / H
    clean_img = np.fft.ifft2(freq_clean_img).real
    clean_img = clean_img[0:m,0:n]
    clean_img_disp = np.uint8(cv2.normalize(clean_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/filtered_image_{r}.png',clean_img_disp)
    return clean_img


img1 = cv2.imread('temple.png', cv2.IMREAD_GRAYSCALE)


img2 = degrade_image(img1, 8)

img3 = inverse_filtering(img2, 8)


In [ ]:

a, b = 256, 256
sigma = 1.0 

n = np.random.normal(0, sigma, (a, b))
N = np.fft.fft2(n)

ps = np.abs(N)**2

th_Sn = sigma**2 * a * b

print(f"Theoretical value of S_n: {th_Sn}")
print(f"Numerical average of |N(u,v)|^2: {np.mean(ps)}")


In [ ]:
def degrade_and_weiner_filtering_1(img, r):
    bl_img = apply_freq_domain_blurring(img, r)
    stddev = 1
    noise = np.random.normal(0, stddev, img.shape)

    d_img= bl_img + noise

    d_img_disp = np.uint8(cv2.normalize(d_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/degraded_image_weiner_1_{r}.png',d_img_disp)

    m,n = img.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_d_img = pad_image(d_img)
    padded_img = pad_image(img)

    freq_d_img = np.fft.fft2(padded_d_img)
    freq_img = np.fft.fft2(padded_img)
    freq_img[freq_img<1e-6] = 1e-6

    s = stddev**2 *m*n

    H = np.fft.fft2(kernel)
    H[H<1e-6] = 1e-6
    mod_H_2 =np.abs(H)**2
    W = (1/H) * (mod_H_2/ (mod_H_2+ (s/freq_img)))

    W_disp = np.uint8(cv2.normalize(np.abs(W), None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/W_weiner_1_{r}.png',W_disp)

    F_hat = W * freq_d_img
    clean_img = np.fft.ifft2(F_hat).real
    clean_img = clean_img[0:m,0:n]
    clean_img_disp = np.uint8(cv2.normalize(clean_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/filtered_image_weiner_1_{r}.png',clean_img_disp)
    return clean_img

degrade_and_weiner_filtering_1(img1,8)

In [ ]:
import matplotlib.pyplot as plt

def degrade_and_weiner_filtering_2(img, r, k):
    bl_img = apply_freq_domain_blurring(img, r)
    stddev = 1
    noise = np.random.normal(0, stddev, img.shape)

    d_img= bl_img + noise

    d_img_disp = np.uint8(cv2.normalize(d_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/degraded_image_weiner_2_{r}.png',d_img_disp)

    m,n = img.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_d_img = pad_image(d_img)

    freq_d_img = np.fft.fft2(padded_d_img)

    H = np.fft.fft2(kernel)
    H[H<1e-6] = 1e-6
    mod_H_2 =np.abs(H)**2
    W = (1/H) * (mod_H_2/ (mod_H_2+ k))

    W_disp = np.uint8(cv2.normalize(np.abs(W), None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/W_weiner_2_{r}.png',W_disp)

    F_hat = W * freq_d_img
    clean_img = np.fft.ifft2(F_hat).real
    clean_img = clean_img[0:m,0:n]
    clean_img_disp = np.uint8(cv2.normalize(clean_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/filtered_image_weiner_2_{r}.png',clean_img_disp)
    return clean_img

Ks = np.logspace(-1, 2, 100)
mse_values = []
min_mse_value  = np.inf
k_min = 0
m,n = img1.shape[:2]
for k in Ks:
    restored_img = degrade_and_weiner_filtering_2(img1,8, k)
    mse = np.sum((img1-restored_img)**2) / (m*n)
    mse_values.append(mse)
    if(mse < min_mse_value):
        min_mse_value = mse
        k_min = k
    print(k, mse)

degrade_and_weiner_filtering_2(img1, 8, k_min)

plt.figure(figsize=(8, 6))
plt.plot(Ks, mse_values, marker='o')
plt.xscale('log')  # Log scale for K values
plt.xlabel('K (constant)')
plt.ylabel('MSE')
plt.title('MSE vs K in Wiener Filtering')
plt.grid(True)
plt.show()

print(f'k  = {k_min} has minimum MSE = {min_mse_value}')



In [ ]:
def weiner_filtering_with_different_h(d_img, r, k):
    m,n = d_img.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_d_img = pad_image(d_img)

    freq_d_img = np.fft.fft2(padded_d_img)

    s = stddev**2 *m*n

    H = np.fft.fft2(kernel)
    H[H<1e-6] = 1e-6
    mod_H_2 =np.abs(H)**2
    W = (1/H) * (mod_H_2/ (mod_H_2+ k))

    W_disp = np.uint8(cv2.normalize(np.abs(W), None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/W_weiner_3_{r}.png',W_disp)

    F_hat = W * freq_d_img
    clean_img = np.fft.ifft2(F_hat).real
    clean_img = clean_img[0:m,0:n]
    clean_img_disp = np.uint8(cv2.normalize(clean_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part4/filtered_image_weiner_3_{r}.png',clean_img_disp)
    return clean_img

bl_img = apply_freq_domain_blurring(img1, 8)
stddev = 1
noise = np.random.normal(0, stddev, img1.shape)

d_img= bl_img + noise

d_img_disp = np.uint8(cv2.normalize(d_img, None, 0, 255, cv2.NORM_MINMAX))
cv2.imwrite(f'outputs/part4/degraded_image_weiner_3_{8}.png',d_img_disp)

Ks = np.logspace(-1, 2, 100)
r = 6
k_min = 0
min_mse_value = np.inf
m,n = img1.shape[:2]
for k in Ks:
    restored_img = weiner_filtering_with_different_h(d_img, r, k)
    mse = np.sum((img1-restored_img)**2) / (m*n)
    if(mse < min_mse_value):
        min_mse_value = mse
        k_min = k
    print(k, mse)

weiner_filtering_with_different_h(d_img, r, k_min)
print(f'k  = {k_min} has minimum MSE = {min_mse_value}')




In [ ]:
Ks = np.logspace(-1, 2, 100)
r = 12
k_min = 0
min_mse_value = np.inf
m,n = img1.shape[:2]
for k in Ks:
    restored_img = weiner_filtering_with_different_h(d_img, r, k)
    mse = np.sum((img1-restored_img)**2) / (m*n)
    if(mse < min_mse_value):
        min_mse_value = mse
        k_min = k
    print(k, mse)

weiner_filtering_with_different_h(d_img, r, k_min)
print(f'k  = {k_min} has minimum MSE = {min_mse_value}')